In [4]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-sztilko-py')

In [7]:
# információ a területi megnevezések megállapításához : https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level2
megye = 'Gyor-moson-sopron'
# megye = 'Fejer'

startDate = '2018-01-01'
endDate = '2023-01-01'

#hőmérséklet skála megadása
Tmin=15
Tmax=40

In [8]:
## országos hőtérkép generálása
# aoi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Hungary'))

## megyei hőtérkép generálása
aoi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(ee.Filter.eq('ADM1_NAME', megye))

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True)

def apply_cloud_mask_lst(image):
  qa = image.select('QA_PIXEL')
  mask1 = qa.bitwiseAnd(1<<3).eq(0)
  mask2 = qa.bitwiseAnd(1<<5).eq(0)
  return image.updateMask(mask1.And(mask2))

image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(aoi).filterDate(startDate, endDate).map(apply_scale_factors).map(apply_cloud_mask_lst).mean().clip(aoi);
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

ndviMin = ee.Number(ndvi.reduceRegion(reducer=ee.Reducer.min(),geometry=aoi,scale=30,maxPixels=1e9).values().get(0))
ndviMax = ee.Number(ndvi.reduceRegion(reducer=ee.Reducer.max(),geometry=aoi,scale=30,maxPixels=1e9).values().get(0))

fv = ((ndvi.subtract(ndviMin)).divide(ndviMax.subtract(ndviMin))).pow(ee.Number(2)).rename('FV')
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

thermal = image.select('ST_B10').rename('thermal')

lst = thermal.expression(
  '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15',
   {'TB': thermal.select('thermal'),'em': em}).rename('LST hun')

colors = ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
          '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
          '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
          'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
          'ff0000', 'de0101', 'c21301', 'a71001', '911003']

visualization_LST = {
    'band' :['thermal'],
    'min': Tmin,
    'max': Tmax,
    'palette':colors
}

# lokális átlagnál +5°C-kal melegebb pixelek kinyerése ( "forró pontok")
focalMean = lst.focal_mean(2500, 'square', 'meters')
T_peaks = lst.gt(focalMean.add(5.0)).selfMask();

m = geemap.Map()
m.centerObject(aoi, 10)
m.add_basemap('HYBRID')
m.addLayer(T_peaks,{'palette':['black', 'red']})
m.addLayer(lst, visualization_LST)
m.add_colorbar_branca(colors=colors,vmin=Tmin,vmax=Tmax)
display(m)

Map(center=[47.656383489806075, 17.261643118807072], controls=(WidgetControl(options=['position', 'transparent…